In [1]:
# Required Imports
from battle_simulation.Battle import Battle, experiment_winner
from battle_simulation.Pokemon import Pokemon
from ipywidgets import *
from IPython.display import display, clear_output
import os
from pathlib import Path
import pandas as pd
import json
import random
import logging
from traitlets import traitlets

In [37]:
# New child class for user input being from Jupyter widgets instead of just user input

class UserBattleJupyter(Battle):
    def __init__(self, Pokemon1, Pokemon2):
        super().__init__(Pokemon1, Pokemon2)
        
    def battle_execute_turn_user_input(self):
        # overwrite this
        pass

In [38]:
# Get the logger
logger = logging.getLogger()
logger.setLevel(logging.INFO)

mfs_path = os.path.join(str(Path(os.path.realpath('__file__')).parents[3]), 'mfs')
pokemon_df = pd.read_csv(os.path.join(mfs_path, 'pokedex_data.csv'), index_col=2)
pokemon_df.drop('Unnamed: 0', axis=1, inplace=True)

status_effect_df = pd.read_csv(os.path.join(mfs_path, 'status_effects.csv'), index_col=0)

moveset_json = os.path.join(mfs_path, 'moveset.json')
with open(moveset_json) as json_file:
    moveset_data = json.load(json_file)

# First Pokemon, user chooses from input
first_mon_name = input('Pokemon to use from the following options: {}'.format(list(moveset_data.keys())))
first_mon_moveset = moveset_data['Mewtwo']
first_mon = Pokemon('Mewtwo', pokemon_df, first_mon_moveset, status_effect_df)

# Randomize the second one from available choices from moveset dict
second_mon_name = random.choice(list(moveset_data.keys()))
second_mon_moveset = moveset_data[second_mon_name]
second_pokemon = Pokemon(second_mon_name, pokemon_df, second_mon_moveset, status_effect_df)

battle = UserBattleJupyter(first_mon, second_pokemon)
battle.execute_battle()

Pokemon to use from the following options: ['Mewtwo', 'Mew', 'Charizard', 'Blastoise', 'Pikachu']Mew


INFO:root:>>> Battle started between: Mewtwo and Mew
INFO:root:>>> Mewtwo used Psybeam!
INFO:root:>>> Mew HP is now 62.5.
INFO:root:>>> Mew used Mega Punch!
INFO:root:>>> Mewtwo HP is now 61.0.
INFO:root:>>> Mewtwo used Psybeam!
INFO:root:>>> Mew HP is now 25.0.
INFO:root:>>> Mew used Pound!
INFO:root:>>> Mewtwo HP is now 36.0.
INFO:root:>>> Mewtwo used Psychic!
INFO:root:>>> Mew has fainted.
INFO:root:>>> Mewtwo is the winner.


'Mewtwo'

In [26]:
# Required functions
global items

def on_button_clicked(b):
    with output:
        clear_output() # clear existing output first to not write another line
        
        for i in range(len(items)):
            items[i].value = False
            
        print("{} was used.".format(b.description))
        b.value = b.description
        
class LoadedButton(widgets.Button):
    """A button that can holds a value as a attribute."""
    # from https://stackoverflow.com/questions/45188464/return-output-of-the-function-executed-on-click

    def __init__(self, value=None, *args, **kwargs):
        super(LoadedButton, self).__init__(*args, **kwargs)
        # Create the value attribute.
        self.add_traits(value=traitlets.Any(value))

In [27]:
# Widgets to be displayed for the move options

words = [list(first_mon_moveset[k].keys())[0] for k in first_mon_moveset]
global items
items = [LoadedButton(description=w) for w in words]
left_box = VBox([items[0], items[1]])
right_box = VBox([items[2], items[3]])
HBox([left_box, right_box])

output = widgets.Output()
print('Choose the move that {name} will use from the following: '.format(name=first_mon.name))
display(HBox([left_box, right_box]), output)

items[0].on_click(on_button_clicked)
items[1].on_click(on_button_clicked)
items[2].on_click(on_button_clicked)
items[3].on_click(on_button_clicked)


Choose the move that Mewtwo will use from the following: 


Output()

In [29]:
print([items[0].value, items[1].value, items[2].value, items[3].value])

[False, False, 'Psybeam', False]


SyntaxError: unexpected EOF while parsing (<ipython-input-32-20e112d4f915>, line 6)